# Fine Tuning Training

## Install Coqui TTS

In [1]:
# upgrade pip, install tts, and upgrade tensorflow
! pip install -U pip
! pip install TTS
! pip install -U tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.1 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 83.7 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 96.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 48.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 83.6 MB/s eta 0:00:0000:0100:01
  Preparin

## Imports

In [29]:
# import the necessary libraries
import os
import glob
import IPython

from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.tts.datasets import load_tts_samples
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.tts.configs.shared_configs import BaseDatasetConfig

## Pretrained Model

In [3]:
# install package needed for the tts model to work
! sudo apt-get install -y espeak-ng

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 75 not upgraded.
Need to get 4215 kB of archives.
After this operation, 12.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcaudio0 amd64 1.1-4 [7908 B]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libsonic0 amd64 0.2.0-8 [13.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 espeak-ng-data amd64 1.50+dfsg-6 [3682 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libespeak-ng1 amd64 1.50+dfsg-6 [189 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/universe amd64 espeak-ng amd64 1.50+dfsg-6 [322 kB]
Fetched 4215 kB in 1s (4677 kB/s)   
Selecting prev

In [4]:
# download the pretrained model, using a sample text and defining
# a speaker id for the multispeaker model
! tts --text "Downloading the pretrained model." \
      --model_name tts_models/en/vctk/vits \
      --speaker_idx p225

 > Downloading model to /root/.local/share/tts/tts_models--en--vctk--vits
 98%|██████████████████████████████████████ | 144M/148M [00:02<00:00, 53.7MiB/s] > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:2

In [8]:
# list the downloaded model files in the directory
! ls /root/.local/share/tts/tts_models--en--vctk--vits

config.json  model_file.pth  speaker_ids.json


## Load Dataset

In [9]:
# set output path
output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

# set the dataset configuration
# ljspeech dataset was chosen, but only one speaker (002) will be used
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="/kaggle/input/ljspeech-002/metadata.csv", path="/kaggle/input/ljspeech-002"
)

## Config

In [10]:
# model architecture and configuration
# 50 epochs will be trained with 32 training batch size
# learning rate lowered to 0.0001 for fine-tuning
audio_config = VitsAudioConfig(
    sample_rate=22050,
    win_length=1024,
    hop_length=256,
    num_mels=80,
    mel_fmin=0,
    mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech_finetune",
    batch_size=32,
    eval_batch_size=16,
    batch_group_size=5,
    num_loader_workers=8,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=50,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
    lr=0.0001
)

## Audio Processor

In [11]:
# audio processor for feature extraction and audio input/output
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


## Tokenizer

In [12]:
# tokenizer for converting text to sequences of token ids
tokenizer, config = TTSTokenizer.init_from_config(config)

## Load Data Samples

In [13]:
# load the training and evaluation samples from the dataset
# each sample is a list of [text, audio_file_path, speaker_name]
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 337 files in /kaggle/input/ljspeech-002


In [14]:
# model initialization with configuration, audio processor, tokenizer, and no speaker manager
model = Vits(config, ap, tokenizer, speaker_manager=None)

# trainer generic api for training the model
# here, we also restore the pretrained model and start fine tuning
trainer = Trainer(
    TrainerArgs(restore_path="/root/.local/share/tts/tts_models--en--vctk--vits/model_file.pth"),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 4
 | > Num. of Torch Threads: 2
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=tts_train_dir/vits_ljspeech_finetune-June-21-2024_09+44PM-0000000
 > Restoring from model_file.pth ...
 > Restoring Model...
 > Partial model initialization...
 | > Layer missing in the model definition: emb_g.weight
 | > Layer missing in the model definition: posterior_encoder.enc.in_layers.0.weight_g
 | > Layer missing in the m

In [15]:
# start the training process
trainer.fit()


 > EPOCH: 0/50
 --> tts_train_dir/vits_ljspeech_finetune-June-21-2024_09+44PM-0000000


[*] Pre-computing phonemes...


 61%|██████    | 203/334 [00:06<00:04, 32.71it/s]

ðə lˈæɾɚ tˈuː bˌiːɪŋ ˈɔːlsoʊ ɐ pɹˈɪzən fɔːɹ fˈɛlənz ænd vˈeɪɡɹənts ɚɹˈɛstᵻd wɪðˌɪn sˈɜːʔn̩ lˈɪmɪts.,,,,,,,,,,,
 [!] Character '̩' not found in the vocabulary. Discarding it.


100%|██████████| 334/334 [00:10<00:00, 31.70it/s]


...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.28702831268310547 (-0.006515979766845703)
     | > avg_loss_disc: 2.900740146636963 (+0.1516575813293457)
     | > avg_loss_disc_real_0: 0.16303333640098572 (+0.002346336841583252)
     | > avg_loss_disc_real_1: 0.28557682037353516 (+0.04817962646484375)
     | > avg_loss_disc_real_2: 0.24273481965065002 (-0.0007546097040176392)
     | > avg_loss_disc_real_3: 0.25052115321159363 (+0.001177668571472168)
     | > avg_loss_disc_real_4: 0.3121476173400879 (+0.044408708810806274)
     | > avg_loss_disc_real_5: 0.2660349905490875 (+0.012278616428375244)
     | > avg_loss_0: 2.900740146636963 (+0.1516575813293457)
     | > avg_loss_gen: 1.6469101905822754 (-0.06543529033660889)
     | > avg_loss_kl: 2.565465211868286 (+0.3112828731536865)
     | > avg_loss_feat: 0.7763609886169434 (-0.36226701736450195)
     | > avg_loss_mel: 19.073396682739258 (-0.1799755096435547)
     | > avg_loss_duration: 1.5150096416473389 (-0.007645845413208008)
     

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.2931082248687744 (+0.006079912185668945)
     | > avg_loss_disc: 2.915459632873535 (+0.014719486236572266)
     | > avg_loss_disc_real_0: 0.2646695077419281 (+0.10163617134094238)
     | > avg_loss_disc_real_1: 0.28635352849960327 (+0.0007767081260681152)
     | > avg_loss_disc_real_2: 0.25925302505493164 (+0.016518205404281616)
     | > avg_loss_disc_real_3: 0.3027876615524292 (+0.05226650834083557)
     | > avg_loss_disc_real_4: 0.3173648715019226 (+0.005217254161834717)
     | > avg_loss_disc_real_5: 0.31851285696029663 (+0.052477866411209106)
     | > avg_loss_0: 2.915459632873535 (+0.014719486236572266)
     | > avg_loss_gen: 1.9279940128326416 (+0.2810838222503662)
     | > avg_loss_kl: 2.1148970127105713 (-0.45056819915771484)
     | > avg_loss_feat: 0.856626570224762 (+0.0802655816078186)
     | > avg_loss_mel: 20.21913719177246 (+1.1457405090332031)
     | > avg_loss_duration: 1.5135136842727661 (-0.001495957374572754)
     |

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.2888026237487793 (-0.004305601119995117)
     | > avg_loss_disc: 2.887284994125366 (-0.028174638748168945)
     | > avg_loss_disc_real_0: 0.22588634490966797 (-0.03878316283226013)
     | > avg_loss_disc_real_1: 0.2592792809009552 (-0.02707424759864807)
     | > avg_loss_disc_real_2: 0.3070552349090576 (+0.04780220985412598)
     | > avg_loss_disc_real_3: 0.24967201054096222 (-0.05311565101146698)
     | > avg_loss_disc_real_4: 0.2611086964607239 (-0.05625617504119873)
     | > avg_loss_disc_real_5: 0.23603639006614685 (-0.08247646689414978)
     | > avg_loss_0: 2.887284994125366 (-0.028174638748168945)
     | > avg_loss_gen: 1.6858785152435303 (-0.24211549758911133)
     | > avg_loss_kl: 2.139411211013794 (+0.024514198303222656)
     | > avg_loss_feat: 0.9319995641708374 (+0.07537299394607544)
     | > avg_loss_mel: 19.579309463500977 (-0.6398277282714844)
     | > avg_loss_duration: 1.5070953369140625 (-0.006418347358703613)
     | 